In [ ]:
from classy import Class
import numpy as np
import scipy
import matplotlib.pylab as plt
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
import time as time
from scipy import integrate
import pysz_gal.pysz_gal as szgal
from sklearn.model_selection import train_test_split
from scipy.special import gamma
from fitter import Fitter, get_common_distributions, get_distributions
from colossus.cosmology import cosmology
from colossus.lss import mass_function
import scipy.integrate as integrate
from colossus.lss import bias
from scipy import special
from scipy.special import gamma
from astropy import units as u
from astropy.cosmology import Planck15 as cs
from astropy.cosmology import z_at_value
import warnings
from scipy.stats import chi2
warnings.filterwarnings("ignore")

In [ ]:
h=0.7
rho_c=2.775*10**11*0.7**2
Omega_m=0.3
Omega_lambda=0.7
delta=180
c_HI_0=18
alpha=0.44
M_min=10**11.27
M_0=10**9.52
gamma=1.45
d_h=3000/h

def Cl_2h(l,chi_i,chi_f):
    cosmology.setCosmology('planck15')
    def test(chi):
        z=z_at_value(cs.comoving_distance,chi*u.Mpc)
        dlndz=chi2.pdf(z,88.38,-0.06,0.001)
        dchidz=d_h/np.sqrt(Omega_m*(1+z)**3+Omega_lambda)
        w_g=dlndz/dchidz
        k=l/chi
        
        def F_2_halo_integrand(M):
            n=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM'))*(h**3)
            b=bias.haloBias(M*h, z,mdef='vir',model='tinker10')

            #calculate M_HI
            R_v=(M*3/(4*np.pi)/(rho_c*Omega_m*delta))**(1/3) #virial radius
            c_HI=c_HI_0*(M/10**11)**(-0.109)*4/(1+z)**gamma
            r_s=R_v/c_HI

            M_HI=M_0*(M/M_min)**alpha*np.exp(-M_min/M)

            #u_1=2*(1+k**2*r_s**2)**(-2)
            u_1_1=-12*special.sici(0.75*k*r_s)[1]*np.sin(0.75*k*r_s)
            u_1_2=12*special.sici((0.75+c_HI)*k*r_s)[1]*np.sin(0.75*k*r_s)
            u_1_3=special.sici((1+c_HI)*k*r_s)[1]*(4*k*r_s*np.cos(k*r_s)-12*np.sin(k*r_s))
            u_1_4=special.sici(k*r_s)[1]*(-4*k*r_s*np.cos(k*r_s)+12*np.sin(k*r_s))
            u_1_5=-4*np.sin(c_HI*k*r_s)/(1+c_HI)
            u_1_6=12*np.cos(0.75*k*r_s)*special.sici(0.75*k*r_s)[0]-12*np.cos(k*r_s)*special.sici(k*r_s)[0]
            u_1_7=-4*k*r_s*np.sin(k*r_s)*special.sici(k*r_s)[0]
            u_1_8=-12*np.cos(0.75*k*r_s)*special.sici((0.75+c_HI)*k*r_s)[0]
            u_1_9=special.sici((1+c_HI)*k*r_s)[0]*(12*np.cos(k*r_s)+4*k*r_s*np.sin(k*r_s))

            u_1=-(u_1_1+u_1_2+u_1_3+u_1_4+u_1_5+u_1_6+u_1_7+u_1_8+u_1_9)/(k*r_s)

            rho_0=M_HI/(8*np.pi*r_s**3)

            u_HI=4*np.pi*rho_0*r_s**3*u_1/M_HI

            return n*b*M_HI*u_HI

        def rho_mean_integrand(M):
            n=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM'))*(h**3)
            M_HI=M_0*(M/M_min)**alpha*np.exp(-M_min/M)
            return n*M_HI
        
        F_2_x=np.logspace(np.log10(M_i),np.log10(M_f),num=50)
        
        rho_y=np.array([rho_mean_integrand(mi) for mi in F_2_x])
        F_2_y=np.array([F_2_halo_integrand(mi) for mi in F_2_x])
        
        rho_mean=np.trapz(rho_y,F_2_x)
        F_2_halo=np.trapz(F_2_y,F_2_x)/rho_mean
        
        
        Plin=cosmo.pk_lin(k, z)
        
        return chi**(-2)*w_g**2*Plin*F_2_halo**2
    
    x=np.logspace(np.log10(chi_i),np.log10(chi_f),num=5)
    y=np.array([test(xi) for xi in x])
    
    cl=np.trapz(y,x)
    return cl
    

In [ ]:
def Cl_1h(l,chi_i,chi_f):
    cosmology.setCosmology('planck15')
    def test(chi):
        z=z_at_value(cs.comoving_distance,chi*u.Mpc)
        dlndz=chi2.pdf(z,88.38,-0.06,0.001)
        dchidz=d_h/np.sqrt(Omega_m*(1+z)**3+Omega_lambda)
        w_g=dlndz/dchidz
        k=l/chi
        
        def F_1_halo_integrand(M):
        
            n=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM'))*(h**3)
            #calculate M_HI
            R_v=(M*3/(4*np.pi)*1/(rho_c*Omega_m*delta))**(1/3) #virial radius
            c_HI=c_HI_0*(M/10**11)**(-0.109)*4/(1+z)**gamma
            r_s=R_v/c_HI

            M_HI=M_0*(M/M_min)**alpha*np.exp(-M_min/M)

            #u_1=2*(1+k**2*r_s**2)**(-2)
            u_1_1=-12*special.sici(0.75*k*r_s)[1]*np.sin(0.75*k*r_s)
            u_1_2=12*special.sici((0.75+c_HI)*k*r_s)[1]*np.sin(0.75*k*r_s)
            u_1_3=special.sici((1+c_HI)*k*r_s)[1]*(4*k*r_s*np.cos(k*r_s)-12*np.sin(k*r_s))
            u_1_4=special.sici(k*r_s)[1]*(-4*k*r_s*np.cos(k*r_s)+12*np.sin(k*r_s))
            u_1_5=-4*np.sin(c_HI*k*r_s)/(1+c_HI)
            u_1_6=12*np.cos(0.75*k*r_s)*special.sici(0.75*k*r_s)[0]-12*np.cos(k*r_s)*special.sici(k*r_s)[0]
            u_1_7=-4*k*r_s*np.sin(k*r_s)*special.sici(k*r_s)[0]
            u_1_8=-12*np.cos(0.75*k*r_s)*special.sici((0.75+c_HI)*k*r_s)[0]
            u_1_9=special.sici((1+c_HI)*k*r_s)[0]*(12*np.cos(k*r_s)+4*k*r_s*np.sin(k*r_s))

            u_1=-(u_1_1+u_1_2+u_1_3+u_1_4+u_1_5+u_1_6+u_1_7+u_1_8+u_1_9)/(k*r_s)

            rho_0=M_HI/(8*np.pi*r_s**3)

            u_HI=4*np.pi*rho_0*r_s**3*u_1/M_HI

            return n*(M_HI*u_HI)**2
    
        def rho_mean_integrand(M):
            n=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM'))*(h**3)
            M_HI=M_0*(M/M_min)**alpha*np.exp(-M_min/M)
            return n*M_HI
        
        #rho_mean=integrate.quad(rho_mean_integrand,M_i,M_f)[0]
        
        F_1_x=np.logspace(np.log10(M_i),np.log10(M_f),num=50)
        
        rho_y=np.array([rho_mean_integrand(mi) for mi in F_1_x])
        F_1_y=np.array([F_1_halo_integrand(mi) for mi in F_1_x])
        
        rho_mean=np.trapz(rho_y,F_1_x)
        F_1_halo=np.trapz(F_1_y,F_1_x)
        
        return chi**(-2)*w_g**2*F_1_halo/rho_mean**2
    
    x=np.logspace(np.log10(chi_i),np.log10(chi_f),num=5)
    y=np.array([test(xi) for xi in x])
    
    cl=np.trapz(y,x)
    return cl